In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [9]:
spark = SparkSession.builder.appName("imdb_query_streaming").getOrCreate()

In [10]:
from metadata import raw

spark.conf.set("spark.sql.streaming.schemaInference", True)
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", True)

title_akas_sdf = (
    spark.readStream.options(
        **{
            "sep": "\t",
            "header": True,
            "nullValue": r"\N",
            "compression": "gzip",
            "cleanSource": "archive",
            "sourceArchiveDir": "./data/archive",
            "maxFilesPerTrigger": 1,
        }
    )
    # .schema(raw.title_akas.SCHEMA)
    .format("csv").load(f"data/input")
)

(
    title_akas_sdf.limit(1)
    .writeStream.format("console")
    .outputMode("complete")
    .options(
        **{
            # "path": "data/output",
            "checkpointLocation": "checkpoint_dir"
        }
    )
    .start()
    .awaitTermination()
)

24/10/11 18:23:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


AnalysisException: Complete output mode not supported when there are no streaming aggregations on streaming DataFrames/Datasets;
GlobalLimit 1
+- LocalLimit 1
   +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@7a43a770,csv,List(),None,List(),None,Map(cleanSource -> archive, path -> data/input, maxFilesPerTrigger -> 1, compression -> gzip, nullValue -> \N, sourceArchiveDir -> ./data/archive, sep -> 	, header -> true),None), FileSource[data/input], [titleId#231, ordering#232, title#233, region#234, language#235, types#236, attributes#237, isOriginalTitle#238]


24/10/11 19:24:34 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 6, schema size: 8
CSV file: file:///workspaces/Streaming-Playground/data/input/name.basics.tsv.gz
24/10/11 19:24:35 WARN HDFSBackedStateStoreProvider: The state for version 3 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
24/10/11 19:24:35 ERROR Executor: Exception in task 0.0 in stage 5.0 (TID 5)
java.lang.IllegalStateException: Error reading delta file file:/workspaces/Streaming-Playground/checkpoint_dir/state/0/0/1.delta of HDFSStateStoreProvider[id = (op=0,part=0),dir = file:/workspaces/Streaming-Playground/checkpoint_dir/state/0/0]: file:/workspaces/Streaming-Playground/checkpoint_dir/state/0/0/1.delta does not exist
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider.updateFromDeltaFile(HDFSBackedStateStoreProvider.scala:464

In [8]:
# from pathlib import Path
# from metadata import raw

# imdb_tables = {}
# data_folder = Path("./data")

# for file in list(data_folder.glob("*.tsv.gz")):
#     table_name = "_".join(file.name.split(".")[:2])
#     print(table_name)
#     imdb_tables[table_name] = (
#         spark.readStream.options(
#             **{
#                 "sep": "\t",
#                 "header": True,
#                 "compression": "gzip",
#                 "nullValue": r"\N",
#             }
#         )
#         .schema(raw.title_akas.SCHEMA)
#         .csv(f"./data/{file.name}")
#         # .limit(100)
#     )

title_akas
title_principals
title_ratings
title_basics
name_basics
title_episode
title_crew
